In [1]:
import numpy as np
# import seaborn as sns
import pandas as pd
import os.path

import matplotlib.pyplot as plt

import tifffile 
import czifile

from skimage import transform
from scipy import ndimage

import random 
import math

In [2]:
from skimage.registration import optical_flow_tvl1
from skimage.morphology import remove_small_objects
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb
from skimage.filters import (threshold_otsu, threshold_niblack,
                             threshold_sauvola)

from skimage.morphology import binary_opening, binary_dilation
from skimage.morphology import disk


from scipy import ndimage
from scipy.ndimage import distance_transform_cdt
from scipy.ndimage import gaussian_filter

from utils.pre_processing_utils import intensity_normalization

In [ ]:
patch_size = int(32)
movie_name = 'pFAK_20230423'

####################
root_processed_data_dir = '/mnt/d/lding/FA/analysis_results/encoder/processed_data'
root_partitioned_data_dir = '/mnt/d/lding/FA/analysis_results/encoder/partitioned_data_rand_flow'
root_plot_dir = '/mnt/d/lding/FA/analysis_results/encoder/plot'


if not os.path.isdir(root_processed_data_dir):
    os.makedirs(root_processed_data_dir)
if not os.path.isdir(root_partitioned_data_dir):
    os.makedirs(root_partitioned_data_dir)
if not os.path.isdir(root_plot_dir):
    os.makedirs(root_plot_dir)


processed_data_dir =  os.path.join(root_processed_data_dir, movie_name)

partitioned_data_dir =  os.path.join(root_partitioned_data_dir, movie_name+'_patchsize_'+str(patch_size))
plot_dir =  os.path.join(root_plot_dir, movie_name+'_patchsize_'+str(patch_size))

if not os.path.isdir(processed_data_dir):
    os.makedirs(processed_data_dir)
if not os.path.isdir(partitioned_data_dir):
    os.makedirs(partitioned_data_dir)
if not os.path.isdir(plot_dir):
    os.makedirs(plot_dir)



In [ ]:
czimovie_dir =  '/mnt/d/lding/zyxin/data/240423_fixed_EGFP_zyxin_pFAK/Control_images'

csv_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/control_pFak/Vessp555-LocLThrk1_dot_csv'
plot_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/control_pFak/Vessp555-LocLThrk1_dot_plot'
seg_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/control_pFak/Vessp555-LocLThrk1_dot_seg'

pixel_size = 0.0706
time_point = 0

if not os.path.isdir(csv_output_dir):
    os.makedirs(csv_output_dir)
if not os.path.isdir(plot_output_dir):
    os.makedirs(plot_output_dir)
if not os.path.isdir(seg_output_dir):
    os.makedirs(seg_output_dir)

In [5]:
def rotate_coor(x_i,y_i,x_c,y_c,rotate_angle):
 
    rotate_angle = rotate_angle*np.pi/180
 
    x_o = (x_i-x_c)*math.cos(rotate_angle) - (2*y_c-y_i-y_c)*math.sin(rotate_angle) +x_c
    y_o = -(x_i-x_c)*math.sin(rotate_angle) - (2*y_c-y_i-y_c)*math.cos(rotate_angle) +(2*y_c-y_c)

    return([x_o,y_o])

In [ ]:
movie_processed_data_dir =  processed_data_dir
movie_partitioned_data_dir =  partitioned_data_dir
movie_plot_dir =  plot_dir


movie_label_output_dir = os.path.join(seg_output_dir,  'label')
movie_mask_output_dir = os.path.join(seg_output_dir,  'mask')

filenames = [x for x in os.listdir(czimovie_dir) if os.path.isfile(os.path.join(czimovie_dir, x)) and ('.tif' in x)]


In [19]:
for filenameID in range(52,len(filenames),2):
# for filenameID in range(1,2):
    
    filename = filenames[filenameID]
    img = czifile.imread(os.path.join(czimovie_dir,filename))
    pax_img = img[0,0,0,:,:,:,0].squeeze()
    actin_img = img[0,0,-1,:,:,:,0].squeeze()
    
    MIP_pax_ori_img = pax_img.max(axis=0)
    intensity_scaling_param = [5,25]
    norm_pax_img = intensity_normalization(pax_img, scaling_param=intensity_scaling_param)
    norm_actin_img = intensity_normalization(actin_img, scaling_param=intensity_scaling_param)
     
    MIP_pax_img = norm_pax_img.max(axis=0)
    # mid_ind = min(8, int(norm_pax_img.shape[0]*2/3))
    # mid_pax_img = norm_pax_img[mid_ind,:,:]

    MIP_actin_img = norm_actin_img.max(axis=0)

    smooth_MIP_actin_img = gaussian_filter(MIP_actin_img,sigma=4,mode='nearest',truncate=3)
    smooth_MIP_pax_img = gaussian_filter(MIP_pax_img,sigma=4,mode='nearest',truncate=3)

    ### low threshold to get cell masks
    new_cell_mask = tifffile.imread(os.path.join(movie_mask_output_dir, 'SS_cell_mask_'+filename+'_MIP'+'.tif')) 
    
    label_pax_seg = tifffile.imread(os.path.join(movie_label_output_dir, 'SS_pax_seglabel_'+filename+'_MIP'+'.tif'))   

    prop_df_pax =  pd.read_csv(os.path.join(csv_output_dir,'labelprop_SS_'+filename+'_MIP'+'.csv'))
    
    for obj_ID in range(0,len(prop_df_pax)):
        area = prop_df_pax.iloc[obj_ID]['area']

        if area <= 10 :
            continue

        orientation = prop_df_pax.iloc[obj_ID]['orientation']
        cell_edge_orientation = prop_df_pax.iloc[obj_ID]['cell_edge_orient']
        label_ID = prop_df_pax.iloc[obj_ID]['label']

        seg = label_pax_seg==label_ID

        for_display_seg = (label_pax_seg>0).astype(np.float32) + (label_pax_seg==label_ID).astype(np.float32)

        regionprops_pax = regionprops(label(label_pax_seg==label_ID))
        x_c = round(regionprops_pax[0].centroid[1])
        y_c = round(regionprops_pax[0].centroid[0])

        y_left = y_c - patch_size
        x_left = x_c - patch_size

        y_right = y_c + patch_size
        x_right = x_c + patch_size


        print(filename + '_'+ str(y_left)+ '_'+ str(x_left)+ '_'+ str(y_right)+ '_'+ str(x_right))

        if y_left < 0 or x_left < 0 or y_right >= pax_img.shape[1] or x_right >= pax_img.shape[2]:
            continue

        patch_img = MIP_pax_img[y_left:y_right,x_left:x_right]
        patch_seg = for_display_seg[y_left:y_right,x_left:x_right]
        
        rand_angle = random.random() * 0 - orientation*180/(np.pi)
        rand_tx = int(random.random() * 16 - 8) * 0
        rand_ty = int(random.random() * 16 - 8) * 0
        
        rotated_patch_img = transform.rotate(patch_img, rand_angle, resize=False, center=None, order=None, mode='constant', cval=0, clip=True)
        rotated_patch_seg= transform.rotate(patch_seg, rand_angle, resize=False, center=None, order=None, mode='constant', cval=0, clip=True)


        cx_left = patch_size/2 - rand_tx
        cx_right = patch_size + patch_size/2 - rand_tx
        cy_up = patch_size/2 -rand_ty
        cy_down = patch_size + patch_size/2-rand_ty

        crop_patch_img = rotated_patch_img[round(patch_size/2-rand_ty):round(patch_size+patch_size/2-rand_ty),round(patch_size/2-rand_tx):round(patch_size+patch_size/2-rand_tx)]
        
        crop_img_filename = filename[:-4]+'_cell'+str(label_ID).zfill(4)+'x'+str(x_c).zfill(4)+'y'+str(y_c).zfill(4)+'ps'+str(patch_size)+'.tif'
        # print(crop_img_filename)
        tifffile.imsave(os.path.join(movie_partitioned_data_dir,crop_img_filename), crop_patch_img)

        X_TransRotCrop = np.array([cx_left, cx_left, cx_right, cx_right, cx_left])
        Y_TransRotCrop = np.array([cy_up, cy_down, cy_down, cy_up, cy_up])

        [X_bigger_patch, Y_bigger_patch] = rotate_coor(X_TransRotCrop,Y_TransRotCrop,patch_size,patch_size,-rand_angle)

        fig, ax = plt.subplots(2,3, figsize=(12,8), dpi=256, facecolor='w', edgecolor='k')
        ax[0,0].imshow(smooth_MIP_pax_img, cmap=plt.cm.gray,vmax=1,vmin=0)
        ax[0,0].plot([x_left, x_left, x_right, x_right, x_left],[y_left,y_right, y_right, y_left,y_left],color='r')
        ax[0,0].plot(X_bigger_patch+x_left, Y_bigger_patch+y_left,color='green')                        

        ax[0,1].imshow(for_display_seg, cmap=plt.cm.gray,vmax=2,vmin=0)
        ax[0,1].plot([x_left, x_left, x_right, x_right, x_left],[y_left,y_right, y_right, y_left,y_left],color='r')
        ax[0,1].plot(X_bigger_patch+x_left, Y_bigger_patch+y_left,color='green')

        ax[0,2].imshow(patch_img, cmap=plt.cm.gray,vmax=1,vmin=0)
        ax[0,2].plot(X_bigger_patch, Y_bigger_patch,color='green')

      
        
        ax[1,0].imshow(rotated_patch_img, cmap=plt.cm.gray,vmax=1,vmin=0)
        ax[1,0].plot([patch_size/2-rand_tx, patch_size/2-rand_tx, patch_size+patch_size/2-rand_tx, patch_size+patch_size/2-rand_tx, patch_size/2-rand_tx],
                     [patch_size/2-rand_ty,patch_size+patch_size/2-rand_ty, patch_size+patch_size/2-rand_ty, patch_size/2-rand_ty,patch_size/2-rand_ty],color='r')
        

        ax[1,1].imshow(rotated_patch_seg, cmap=plt.cm.gray,vmax=2,vmin=0)
        ax[1,1].plot([patch_size/2-rand_tx, patch_size/2-rand_tx, patch_size+patch_size/2-rand_tx, patch_size+patch_size/2-rand_tx, patch_size/2-rand_tx],
                     [patch_size/2-rand_ty,patch_size+patch_size/2-rand_ty, patch_size+patch_size/2-rand_ty, patch_size/2-rand_ty,patch_size/2-rand_ty],color='r')
        
        ax[1,2].imshow(crop_patch_img, cmap=plt.cm.gray,vmax=1,vmin=0)
        
        plot_filename = filename[:-4]+'_label'+str(label_ID).zfill(4)+'_xc'+str(x_c)+'_yc'+str(y_c)+ '_fa_orient.png'
        fig.savefig(os.path.join(movie_plot_dir,plot_filename),bbox_inch='tight')
        # print(plot_filename)
        plt.close(fig) 


Rock 24hr 5.czi_-60_1715_68_1843
Rock 24hr 5.czi_-24_1735_104_1863
Rock 24hr 5.czi_14_996_142_1124


/home/ldin/anaconda3/envs/segmentation/lib/python3.7/site-packages/ipykernel_launcher.py:113: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "bbox_inch" which is no longer supported as of 3.3 and will become an error in 3.6


Rock 24hr 5.czi_15_1018_143_1146
Rock 24hr 5.czi_24_1021_152_1149
Rock 24hr 5.czi_52_971_180_1099
Rock 24hr 5.czi_68_1052_196_1180
Rock 24hr 5.czi_106_1099_234_1227
Rock 24hr 5.czi_118_1650_246_1778
Rock 24hr 5.czi_118_955_246_1083
Rock 24hr 5.czi_126_1017_254_1145
Rock 24hr 5.czi_182_1750_310_1878
Rock 24hr 5.czi_183_1155_311_1283
Rock 24hr 5.czi_188_1751_316_1879
Rock 24hr 5.czi_186_825_314_953
Rock 24hr 5.czi_203_1627_331_1755
Rock 24hr 5.czi_220_793_348_921
Rock 24hr 5.czi_240_766_368_894
Rock 24hr 5.czi_243_1229_371_1357
Rock 24hr 5.czi_274_1634_402_1762
Rock 24hr 5.czi_296_1193_424_1321
Rock 24hr 5.czi_293_1262_421_1390
Rock 24hr 5.czi_306_1838_434_1966
Rock 24hr 5.czi_310_930_438_1058
Rock 24hr 5.czi_323_1278_451_1406
Rock 24hr 5.czi_336_904_464_1032
Rock 24hr 5.czi_358_1809_486_1937
Rock 24hr 5.czi_379_1302_507_1430
Rock 24hr 5.czi_377_1748_505_1876
Rock 24hr 5.czi_384_1799_512_1927
Rock 24hr 5.czi_394_945_522_1073
Rock 24hr 5.czi_406_1033_534_1161
Rock 24hr 5.czi_413_1661_541_

: 

In [17]:
len(filenames)

56

In [16]:
filename

'Control 2.czi'

In [10]:
pax_img.shape

(9, 1908, 1908)

In [11]:
img.shape

(1, 1, 3, 9, 1908, 1908, 1)

In [12]:
patch_size/2

32.0

In [13]:

        #         y_c = int(y_0+(y_i-0.5)*patch_size)
        #         x_c = int(x_0+(x_i-0.5)*patch_size)

        #         y_left = y_c - patch_size
        #         x_left = x_c - patch_size

        #         y_right = y_c + patch_size
        #         x_right = x_c + patch_size


        #         if y_left < 0 or x_left < 0 or y_right >= img.shape[1] or x_right >= img.shape[2]:
        #             continue

        #         patch_img = train_t0_img[y_left:y_right,x_left:x_right]
                
        #         patch_seg = train_seg[y_left:y_right,x_left:x_right]
        #         rand_angle = random.random() * 0
        #         rand_tx = int(random.random() * 16 - 8)
        #         rand_ty = int(random.random() * 16 - 8)
                

        #         rotated_patch_img = transform.rotate(patch_img, rand_angle, resize=False, center=None, order=None, mode='constant', cval=0, clip=True)
        #         rotated_patch_seg = transform.rotate(patch_seg, rand_angle, resize=False, center=None, order=None, mode='constant', cval=0, clip=True)
                                
        #         cx_left = 16-rand_tx
        #         cx_right = 32+16-rand_tx
        #         cy_up = 16-rand_ty
        #         cy_down = 32+16-rand_ty

        #         crop_patch_img = rotated_patch_img[16-rand_ty:32+16-rand_ty,16-rand_tx:32+16-rand_tx]
        #         crop_patch_seg = rotated_patch_seg[16-rand_ty:32+16-rand_ty,16-rand_tx:32+16-rand_tx]

        #         if((crop_patch_seg.mean())<0.95):
        #             continue

        #         crop_img_filename = 't'+str(frame_ID).zfill(4)+'x'+str(x_c).zfill(4)+'y'+str(y_c).zfill(4)+'ps'+str(patch_size)+'.tif'
        #         tifffile.imsave(os.path.join(movie_partitioned_data_dir,crop_img_filename), crop_patch_img)


        #         X_TransRotCrop = np.array([cx_left, cx_left, cx_right, cx_right, cx_left])
        #         Y_TransRotCrop = np.array([cy_up, cy_down, cy_down, cy_up, cy_up])

        #         [X_bigger_patch, Y_bigger_patch] = rotate_coor(X_TransRotCrop,Y_TransRotCrop,32,32,-rand_angle)


        #         fig, ax = plt.subplots(2,4, figsize=(16,8), dpi=256, facecolor='w', edgecolor='k')
        #         ax[0,0].imshow(train_img, cmap=plt.cm.gray,vmax=1,vmin=0)
        #         ax[0,0].plot([x_left, x_left, x_right, x_right, x_left],[y_left,y_right, y_right, y_left,y_left],color='r')
        #         ax[0,0].plot(X_bigger_patch+x_left, Y_bigger_patch+y_left,color='green')

        #         ax[0,1].imshow(train_seg, cmap=plt.cm.gray,vmax=1,vmin=0)
        #         ax[0,1].plot([x_left, x_left, x_right, x_right, x_left],[y_left,y_right, y_right, y_left,y_left],color='r')
        #         ax[0,0].plot(X_bigger_patch+x_left, Y_bigger_patch+y_left,color='green')

                

        #         ax[0,2].imshow(patch_img, cmap=plt.cm.gray,vmax=1,vmin=0)
        #         ax[0,2].plot(X_bigger_patch, Y_bigger_patch,color='green')

        #         ax[0,3].imshow(patch_seg, cmap=plt.cm.gray,vmax=1,vmin=0)
        #         ax[0,3].plot(X_bigger_patch, Y_bigger_patch,color='green')

        #         ax[1,0].imshow(rotated_patch_img, cmap=plt.cm.gray,vmax=1,vmin=0)
        #         ax[1,0].plot([16-rand_tx, 16-rand_tx, 32+16-rand_tx, 32+16-rand_tx, 16-rand_tx],[16-rand_ty,32+16-rand_ty, 32+16-rand_ty, 16-rand_ty,16-rand_ty],color='r')

        #         ax[1,1].imshow(rotated_patch_seg, cmap=plt.cm.gray,vmax=1,vmin=0)
        #         ax[1,1].plot([16-rand_tx, 16-rand_tx, 32+16-rand_tx, 32+16-rand_tx, 16-rand_tx],[16-rand_ty,32+16-rand_ty, 32+16-rand_ty, 16-rand_ty,16-rand_ty],color='r')

        #         ax[1,2].imshow(crop_patch_img, cmap=plt.cm.gray,vmax=1,vmin=0)
        #         ax[1,3].imshow(crop_patch_seg, cmap=plt.cm.gray,vmax=1,vmin=0)
                
        #         plot_filename = 'plot_grid_t'+str(frame_ID).zfill(4)+'_xc'+str(x_c)+'_yc'+str(y_c)+ '.png'
        #         fig.savefig(os.path.join(movie_plot_dir,plot_filename),bbox_inch='tight')
        #         plt.close(fig) 
                
        #         ax_accu[0].plot(X_bigger_patch+x_left, Y_bigger_patch+y_left,color='green')
        #         ax_accu[1].plot(X_bigger_patch+x_left, Y_bigger_patch+y_left,color='green')


                
        #         s = pd.Series([movie_dir, filename, frame_ID, x_c,y_c,rand_angle,rand_tx,rand_ty,
        #             X_bigger_patch[0]+x_left,X_bigger_patch[1]+x_left,X_bigger_patch[2]+x_left,X_bigger_patch[3]+x_left,
        #             Y_bigger_patch[0]+y_left,Y_bigger_patch[1]+y_left,Y_bigger_patch[2]+y_left,Y_bigger_patch[3]+y_left,
        #             movie_partitioned_data_dir,crop_img_filename,movie_plot_dir,plot_filename],
        #             index=['movie_dir','filename','frame_ID','x_c','y_c','rand_angle','rand_tx','rand_ty',
        #                                     'x_corner1','x_corner2','x_corner3','x_corner4','y_corner1','y_corner2','y_corner3','y_corner4',
        #                                     'movie_partitioned_data_dir','crop_img_filename','movie_plot_dir','plot_filename'])
                
        #         data_prep_record = data_prep_record.append(s,ignore_index=True)

        # data_prep_record.to_csv(os.path.join(movie_plot_dir,'data_prep_record_'+str(filenameID)+'_t'+str(frame_ID)+'.csv'))

        # fig_accu.savefig(os.path.join(movie_plot_dir,'grid_t'+str(frame_ID).zfill(4)+'.png'),bbox_inch='tight')   
        # plt.close(fig_accu)   
                

In [14]:
len(prop_df_pax)


567